# Lesson 2 Exercise 1: Creating Normalized Tables

<img src="images/postgresSQLlogo.png" width="250" height="250">

## In this exercise we are going to walk through the basics of modeling data in normalized form. We will create tables in PostgreSQL, insert rows of data, and do simple JOIN SQL queries to show how these mutliple tables can work together. 

#### Where you see ##### you will need to fill in code.


#### Import the library 
Note: An error might popup after this command has exectuted. If it does, read it carefully before ignoring. 

In [26]:
import psycopg2
from psycopg2 import sql
import pandas as pd

__Create a connection to the database, get a cursor, and set autocommit to true)__

In [31]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)
conn.set_session(autocommit=True)

#### Let's imagine we have a table called Music Store. 

`Table Name: music_store
column 0: Transaction Id
column 1: Customer Name
column 2: Cashier Name
column 3: Year 
column 4: Albums Purchased`


## Now to translate this information into a CREATE Table Statement and insert the data

<img src="images/table12.png" width="650" height="650">




In [39]:
row_1 = [1, 'Amanda', 'Adam', 2000, ['Rubber Soul', 'Let it Be']]
row_2 = [2, 'Toby', 'Sam', 2000, ['My Generation']]
row_3 = [3, 'Max', 'Bob', 2018, ['Meet the Beatles','Help!']]

In [40]:
data = {'row_1': row_1, 'row_2': row_2, 'row_3': row_3}
df = pd.DataFrame.from_dict(data, orient='index', \
                            columns=['transaction_id','customer_name','cashier_name','year','albums_purchased'])
df

,transaction_id,customer_name,cashier_name,year,albums_purchased
row_1,1,Amanda,Adam,2000,"[Rubber Soul, Let it Be]"
row_2,2,Toby,Sam,2000,[My Generation]
row_3,3,Max,Bob,2018,"[Meet the Beatles, Help!]"


## Functions
this section is functions I made to make this scaleable

In [34]:
# helger function for update row
def build_update_query(table, col_names):
    """
    Assumes you don't want to update primary key at position zero
    input: takes table name as string as well as list of df column names
    output: returns formatted query
    """
    
    place_holders = sql.SQL(',').join(sql.Placeholder() * len(col_names[1:]))
    col_name_identifiers = sql.SQL(',').join([sql.Identifier(str(col_name)) for col_name in col_names[1:]])
    
    
    query = sql.SQL("UPDATE {table_name} SET ({set_cols}) = ({values})").format(
        table_name = sql.Identifier(table),
        set_cols = col_name_identifiers,
        values = place_holders
        )
    return query

# helper function for update_db
def update_row(cur, table, col_names, values):
    
    query = build_update_query(table, col_names)
    
    cur.execute(query, tuple(values)) 
    
# helper function for update_db
def check_if_primary_key_exists(cur, primary_key):
    query = ("""SELECT transaction_id FROM music_store WHERE transaction_id = %s""")
    cur.execute(query, (primary_key,))
    
    return cur.fetchone() is not None


def update_db(cur, df, table, col_names):
    """
    input: cursor, dataframe, table as string, col_names as list 
    output: updates records already in db through update_row function 
    and also returns a temporary df with all records not already stored in db
    """
    
    tmp_df = pd.DataFrame(columns=col_names)

    for i,row in df.iterrows():
        # check to see if the primary key exists
        if check_if_primary_key_exists(cur, row['transaction_id']): # if primary key exists then update
            row_list = [row[col_name] for col_name in col_names[1:]]
            update_row(cur, table, col_names, row_list)
            
        else: # The primary key doesn't exist so append to the db table
            tmp_df = tmp_df.append(row)
    
    return tmp_df

#---------------------------------------------------------------------
# inserting into the database
# --> the following functions help build the insert_into_table function

# helper function for insert_into_table
# builds query "INSERT INTO table (col_names) VALUES(%s, %s...);"
def build_insert_query(table, col_names):
    """
    input: takes table name as string as well as list of df column names
    output: returns formatted query
    """
    place_holders = sql.SQL(',').join(sql.Placeholder() * len(col_names))
    col_names = sql.SQL(',').join([sql.Identifier(str(col_name)) for col_name in col_names])
    
    query = sql.SQL("INSERT INTO {table_name} ({col_names}) VALUES ({values})").format(
        table_name = sql.Identifier(table),
        col_names = col_names,
        values = place_holders
        )
    
    return query

# helper function for append_from_df_to_db
def insert_into_table(cur, table, col_names, values):
    """
    inserts an individual record into table
    """
    query = build_insert_query(table, col_names)
    
    cur.execute(query, tuple(values)) 

# loops over rows in dataframe and inserts into database    
def append_from_df_to_db(cur, df, table, col_names):    
    for i, row in df.iterrows():
        row_list = [row[col_name] for col_name in col_names]
        insert_into_table(cur, table, col_names, row_list)

In [32]:
try: 
    cur.execute("""CREATE TABLE IF NOT EXISTS music_store (transaction_id int
                    , customer_name varchar
                    , cashier_name varchar
                    , year int
                    , albums_purchased text[]);""")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

In [45]:
# if primary key already exists in table this function should update the row, else add row to temp df
tmp_df = update_db(cur, df, 'music_store', df.columns)

In [43]:
# takes rows in temp df and adds them one-by-one to table in db
append_from_df_to_db(cur, tmp_df, 'music_store', df.columns)

In [44]:
try: 
    cur.execute("SELECT * FROM music_store;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(1, 'Max', 'Bob', 2018, ['Meet the Beatles', 'Help!'])
(2, 'Max', 'Bob', 2018, ['Meet the Beatles', 'Help!'])
(3, 'Max', 'Bob', 2018, ['Meet the Beatles', 'Help!'])


In [3]:
# TO-DO: Add the CREATE Table Statement and INSERT statements to add the data in the table
row_1 = [1, 'Amanda', 'Sam', 2000, ['Rubber Soul', 'Let it Be']]
row_2 = [2, 'Toby', 'Sam', 2000, ['My Generation']]
row_3 = [3, 'Max', 'Bob', 2018, ['Meet the Beatles','Help!']]
try: 
    cur.execute("""CREATE TABLE IF NOT EXISTS music_store (transaction_id int
                    , customer_name varchar
                    , cashier_name varchar
                    , year int
                    , albums_purchased text[]);""")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
try: 
    cur.execute("INSERT INTO music_store (transaction_id, customer_name, cashier_name, year, albums_purchased) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (row_1))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO music_store (transaction_id, customer_name, cashier_name, year, albums_purchased) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (row_2))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO music_store (transaction_id, customer_name, cashier_name, year, albums_purchased) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (row_3))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
    
try: 
    cur.execute("SELECT * FROM music_store;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(1, 'Amanda', 'Sam', 2000, ['Rubber Soul', 'Let it Be'])
(2, 'Toby', 'Sam', 2000, ['My Generation'])
(3, 'Max', 'Bob', 2018, ['Meet the Beatles', 'Help!'])


#### Moving to 1st Normal Form (1NF)

### TO-DO: This data has not been normalized. To get this data into 1st normal form, you need to remove any collections or list of data and break up the list of songs into individual rows. 


In [4]:
## TO-DO: Complete the CREATE table statements and INSERT statements

row_1 = [1, 'Amanda', 'Sam', 2000, 'Rubber Soul']
row_2 = [1, 'Amanda', 'Sam', 2000, 'Let it Be']
row_3 = [2, 'Toby', 'Sam', 2000, 'My Generation']
row_4 = [3, 'Max', 'Bob', 2018, 'Meet the Beatles']
row_5 = [3, 'Max', 'Bob', 2018, 'Help!']

try: 
    cur.execute("""CREATE TABLE IF NOT EXISTS music_store2 (
                transaction_id int
                , customer_name varchar
                , cashier_name varchar
                , year int
                , albums_purchased varchar
                );""")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
try: 
    cur.execute("INSERT INTO music_store2 (transaction_id, customer_name, cashier_name, year, albums_purchased) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (row_1))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute("INSERT INTO music_store2 (transaction_id, customer_name, cashier_name, year, albums_purchased) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (row_2))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO music_store2 (transaction_id, customer_name, cashier_name, year, albums_purchased) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (row_3))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO music_store2 (transaction_id, customer_name, cashier_name, year, albums_purchased) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (row_4))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO music_store2 (transaction_id, customer_name, cashier_name, year, albums_purchased) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (row_5))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("SELECT * FROM music_store2;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(1, 'Amanda', 'Sam', 2000, 'Rubber Soul')
(1, 'Amanda', 'Sam', 2000, 'Let it Be')
(2, 'Toby', 'Sam', 2000, 'My Generation')
(3, 'Max', 'Bob', 2018, 'Meet the Beatles')
(3, 'Max', 'Bob', 2018, 'Help!')


#### Moving to 2nd Normal Form (2NF)
You have now moved the data into 1NF, which is the first step in moving to 2nd Normal Form. The table is not yet in 2nd Normal Form. While each of the records in the table is unique, our Primary key (transaction id) is not unique. 

### TO-DO: Break up the table into two tables, transactions and albums sold. 


In [16]:
def drop_table(table):
    try:
        cur.execute("DROP TABLE IF EXISTS {my_table};").format(my_table=sql.Identifier(table))
    except psycopg2.Error as e:
        print("Error: Issue dropping table")
        print(e)

In [8]:
try:
    cur.execute("""DROP TABLE IF EXISTS albums_sold;""")
except psycopg2.Error as e:
    print("Error dropping table")
    print(e)

In [5]:
 
trow_1 = [1, 'Amanda', 'Sam', 2000]
trow_2 = [2, 'Toby', 'Sam', 2000]
trow_3 = [3, 'Max', 'Bob', 2018]

arow_1 = [1, 1, 'Rubber Soul']
arow_2 = [2, 1, 'Let it Be']
arow_3 = [3, 2, 'My Generation']
arow_4 = [4, 3, 'Meet the Beatles']
arow_5 = [5, 3, 'Help!']
try: 
    cur.execute("""CREATE TABLE IF NOT EXISTS transactions (
                transaction_id int
                , customer_name varchar
                , cashier_name varchar
                , year int
                );""")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

try: 
    cur.execute("CREATE TABLE IF NOT EXISTS albums_sold (album_id int, transaction_id int, albums_purchased varchar);")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
try: 
    cur.execute("INSERT INTO transactions (transaction_id, customer_name, cashier_name, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (trow_1))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute("INSERT INTO transactions (transaction_id, customer_name, cashier_name, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (trow_2))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO transactions (transaction_id, customer_name, cashier_name, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (trow_3))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO albums_sold (album_id, transaction_id, albums_purchased) \
                 VALUES (%s, %s, %s)", \
                 (arow_1))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute("INSERT INTO albums_sold (album_id, transaction_id, albums_purchased) \
                 VALUES (%s, %s, %s)", \
                 (arow_2))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO albums_sold (album_id, transaction_id, albums_purchased) \
                 VALUES (%s, %s, %s)", \
                 (arow_3))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO albums_sold (album_id, transaction_id, albums_purchased) \
                 VALUES (%s, %s, %s)", \
                 (arow_4))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute("INSERT INTO albums_sold (album_id, transaction_id, albums_purchased) \
                 VALUES (%s, %s, %s)", \
                 (arow_5))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

print("Table: transactions\n")
try: 
    cur.execute("SELECT * FROM transactions;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

print("\nTable: albums_sold\n")
try: 
    cur.execute("SELECT * FROM albums_sold;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Table: transactions

(1, 'Amanda', 'Sam', 2000)
(2, 'Toby', 'Sam', 2000)
(3, 'Max', 'Bob', 2018)

Table: albums_sold

(1, 1, 'Rubber Soul')
(2, 1, 'Let it Be')
(3, 2, 'My Generation')
(4, 3, 'Meet the Beatles')
(5, 3, 'Help!')


### TO-DO: Do a `JOIN` on these tables to get all the information in the original first Table. 

In [6]:
## TO-DO: Complete the join on the transactions and album_sold tables

try: 
    cur.execute("SELECT * FROM transactions t JOIN albums_sold a ON t.transaction_id = a.transaction_id ;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


(1, 'Amanda', 'Sam', 2000, 1, 1, 'Rubber Soul')
(1, 'Amanda', 'Sam', 2000, 2, 1, 'Let it Be')
(2, 'Toby', 'Sam', 2000, 3, 2, 'My Generation')
(3, 'Max', 'Bob', 2018, 4, 3, 'Meet the Beatles')
(3, 'Max', 'Bob', 2018, 5, 3, 'Help!')


#### Moving to 3rd Normal Form (3NF)
Check our table for any transitive dependencies. 
_HINT:_ Check the table for any transitive dependencies. _Transactions_ can remove _Cashier Name_ to its own table, called _Employees_, which will leave us with 3 tables. 


### TO-DO: Create the third table named *employees* to move to 3rd NF. 


In [21]:
cur.execute("DROP TABLE IF EXISTS employees")

In [22]:

trow_1 =  [1, 1, 'Amanda', 2000]
trow_2 = [2, 1, 'Toby', 2000]
trow_3 = [3, 2, 'Max', 2018]

erow_1 = [1, 'Sam']
erow_2 = [2, 'Bob']
try: 
    cur.execute("""CREATE TABLE IF NOT EXISTS transactions2 (
                transaction_id int
                , employee_id int
                , customer_name varchar
                , year int
                );""")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

try: 
    cur.execute("CREATE TABLE IF NOT EXISTS employees (employee_id int, cashier_name varchar);")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

try: 
    cur.execute("INSERT INTO transactions2 (transaction_id, employee_id, customer_name, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (trow_1))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute("INSERT INTO transactions2 (transaction_id, employee_id, customer_name, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (trow_2))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO transactions2 (transaction_id, employee_id, customer_name, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (trow_3))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute("INSERT INTO employees (employee_id, cashier_name) \
                 VALUES (%s, %s)", \
                 (erow_1))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute("INSERT INTO employees (employee_id, cashier_name) \
                 VALUES (%s, %s)", \
                 (erow_2))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)    

print("Table: transactions2\n")
try: 
    cur.execute("SELECT * FROM transactions2;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

print("\nTable: albums_sold\n")
try: 
    cur.execute("SELECT * FROM albums_sold;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

print("\nTable: employees\n")
try: 
    cur.execute("SELECT * FROM employees;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Table: transactions2

(1, 1, 'Amanda', 2000)
(2, 1, 'Toby', 2000)
(3, 2, 'Max', 2018)

Table: albums_sold

(1, 1, 'Rubber Soul')
(2, 1, 'Let it Be')
(3, 2, 'My Generation')
(4, 3, 'Meet the Beatles')
(5, 3, 'Help!')

Table: employees

(1, 'Sam')
(2, 'Bob')


### TO-DO: Complete the last two `JOIN` on these 3 tables so we can get all the information we had in our first Table. 

In [23]:
try: 
    cur.execute("SELECT * FROM (transactions2 t JOIN employees e ON \
                               t.employee_id = e.employee_id) JOIN \
                               albums_sold a ON a.transaction_id=t.transaction_id;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(1, 1, 'Amanda', 2000, 1, 'Sam', 1, 1, 'Rubber Soul')
(1, 1, 'Amanda', 2000, 1, 'Sam', 2, 1, 'Let it Be')
(2, 1, 'Toby', 2000, 1, 'Sam', 3, 2, 'My Generation')
(3, 2, 'Max', 2018, 2, 'Bob', 4, 3, 'Meet the Beatles')
(3, 2, 'Max', 2018, 2, 'Bob', 5, 3, 'Help!')


### Your output for the above cell should be:

(1, 'Amanda', 1, 2000, 1, 1, 'Rubber Soul', 1, 'Sam')<br>
(1, 'Amanda', 1, 2000, 2, 1, 'Let it Be', 1, 'Sam')<br>
(2, 'Toby', 1, 2000, 3, 2, 'My Generation', 1, 'Sam')<br>
(3, 'Max', 2, 2018, 4, 3, 'Meet the Beatles', 2, 'Bob')<br>
(3, 'Max', 2, 2018, 5, 3, 'Help!', 2, 'Bob')<br>


### Awesome work!! You have Normalized the dataset! 

### And finally close your cursor and connection. 

In [24]:
try: 
    cur.execute("DROP table music_store")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table music_store2")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table albums_sold")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table employees")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table transactions")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table transactions2")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

### And finally close your cursor and connection. 

In [25]:
cur.close()
conn.close()